In [3]:
import pandas as pd
%pip install pyspark
from spark_session_manager import SparkSessionManager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:0000:0100:04
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 673.1 kB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=416dfa4dbe387ad5e16d1bf044663cf6ceb6593053584d2bfe7e38a4b2727744
  Stored in directory: /home/gr00stl/.cache/pip/wheels/95/13/41/f7f135ee114175605fb4f0a89e7389f3742aa6c1e1a5bcb657
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [4]:
df_with_emotions = pd.read_parquet("data/results/emotion-english-distilroberta-base.parquet")
